In [ ]:
import load_utils
import pyEXP
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import astropy.constants as const
import time as t
from astropy.table import Table

start = t.time()  # starting the first timer, start of reading and defining

# define EXP units
expL, expM, expV, expT = load_utils.define_exp_units(M_phys=1.*u.Msun, R_phys=1.*u.kpc, G=const.G.to(u.kpc/(1e10*u.Msun)*u.km**2/u.s**2))
expL = expL.to(u.kpc)
expM = expM.to(u.Msun)
expV = expV.to(u.km/u.s)
expT = expT.to(u.Gyr)

# creating basis configurations
halo_config = """
id: sphereSL
parameters:
  numr : 4000
  rmin : 0.1
  rmax : 500.0
  Lmax : 6
  nmax : 4
  rmapping : 52
  modelname : MW_halo_empirical.txt
  cachename : MW_halo_empirical.cache
"""
halo_basis = pyEXP.basis.Basis.factory(halo_config)

bulge_config = """
id         : sphereSL
parameters :
  numr     : 4000
  rmin     : 0.01
  rmax     : 1000.0 #1000.0 Check max distance
  Lmax     : 0
  nmax     : 10
  rmapping  : 1 #52
  modelname : MW_bulge_empirical.txt
  cachename : MW_bulge.empirical.cache
"""
bulge_basis = pyEXP.basis.Basis.factory(bulge_config)

config_disk_final = """
id: cylinder
parameters:
  acyl: 3.5
  hcyl: 2
  mmax: 2
  rcylmin: 0.001
  rcylmax: 20.0
  nmax: 32  # 8
  nmaxfid: 48  # 24
  ncylnx: 512  # 256
  ncylny: 256  # 128
  lmaxfid: 96  # 24
  ncylodd: 12  # 18
  rnum: 200
  pnum: 1
  tnum: 80
  logr: true
  cachename: eof.cache.teng.file
"""
# config_disk_final = """
# id: cylinder
# parameters:
#   acyl: 3.5
#   hcyl: 2
#   mmax: 2
#   rcylmin: 0.001
#   rcylmax: 20.0
#   nmax: 8
#   nmaxfid: 24
#   ncylnx: 256
#   ncylny: 128
#   lmaxfid: 24
#   ncylodd: 18
#   rnum: 200
#   pnum: 1
#   tnum: 80
#   logr: true
#   cachename: eof.cache.teng.file
# """
disk_basis = pyEXP.basis.Basis.factory(config_disk_final)

sgr_config = """
id: sphereSL
parameters:
  numr : 4000
  rmin : 0.05
  rmax : 150.0
  Lmax : 4
  nmax : 4
  rmapping : 3.4
  modelname : Sgr_empirical.txt
  cachename : Sgr_empirical.cache
"""
sgr_basis = pyEXP.basis.Basis.factory(sgr_config)

halo_coefs = pyEXP.coefs.Coefs.factory('outcoef.MWSgr_hot_halo.h5')
bulge_coefs = pyEXP.coefs.Coefs.factory('outcoef.MWSgr_hot_bulge.h5')
disk_coefs = pyEXP.coefs.Coefs.factory('outcoef.MWSgr_hot_disk.h5')
sgr_coefs = pyEXP.coefs.Coefs.factory('outcoef.MWSgr_hot_sgr.h5')

# scale time data embedded within coefficients
times = halo_coefs.Times()
for time in times:
    con = halo_coefs.getCoefStruct(time)
    #con.time = time/expT  # 'time' variable has unit Gyr (snapnum/100), scaled to expT
    con.setCoefTime(time/expT)
    if time == 0:
        temp = pyEXP.coefs.Coefs.makecoefs(con, 'Halo')
    temp.add(con)
halo_coefs = temp

times = bulge_coefs.Times()
for time in times:
    con = bulge_coefs.getCoefStruct(time)
    #con.time = time/expT
    con.setCoefTime(time/expT)
    if time == 0:
        temp = pyEXP.coefs.Coefs.makecoefs(con, 'Bulge')
    temp.add(con)
bulge_coefs = temp

times = disk_coefs.Times()
for time in times:
    con = disk_coefs.getCoefStruct(time)
    #con.time = time/expT
    con.setCoefTime(time/expT)
    if time == 0:
        temp = pyEXP.coefs.Coefs.makecoefs(con, 'Disk')
    temp.add(con)
disk_coefs = temp

times = sgr_coefs.Times()
for time in times:
    con = sgr_coefs.getCoefStruct(time)
    #con.time = time/expT
    con.setCoefTime(time/expT)
    if time == 0:
        temp = pyEXP.coefs.Coefs.makecoefs(con, 'Halo')
    temp.add(con)
sgr_coefs = temp
    
# define variables and arrays to store data 
nit = 400  # n iterations
id = 49999133  # outer disk star
# 40523293 inner 600
# 41720462 middle 300
# 49517535 inner 600
# 49999133 outer 300

data = Table.read(f'pos_{id}.csv', format='csv')
xpos = data['x'][:nit]  # kpc
ypos = data['y'][:nit]
zpos = data['z'][:nit]
u = data['vx'][0]  # km/s
v = data['vy'][0]
w = data['vz'][0]

begin = t.time()  # end of reading and defining, begin iterations
print('start :',begin-start)

expstart = t.time()  # time the start of EXP reconstruction, iterations here are fast so only one timer
# model = [[disk_basis, disk_coefs], [halo_basis, halo_coefs], [bulge_basis, bulge_coefs], [sgr_basis, sgr_coefs]]  # moving MW + Sgr
model = [[disk_basis, disk_coefs], [halo_basis, halo_coefs], [bulge_basis, bulge_coefs]]  # moving MW + Sgr
phasespace = [[xpos[0]/expL, ypos[0]/expL, zpos[0]/expL, u/expV, v/expV, w/expV]]  # tracer initial conditions, in expL and expV as defined
loop = 1  # split the integration into 10 fractions for efficiency, nit/10 snapshots per fraction
# integrate the orbit: arguments are tinitial,tfinal,tstep,phasespace,model
for i in range(0, nit - 1, 1):
    times, orbits = pyEXP.basis.IntegrateOrbits(loop*i/100/expT, loop*(i + 1)/100/expT, 0.001/100/expT, phasespace, model, pyEXP.basis.AllTimeAccel())  # i/100 (snapnum/100) for Gyr, scaled to expT, 1000 samples per snapshot
    phasespace = [[orbits[0][0][-1], orbits[0][1][-1], orbits[0][2][-1], orbits[0][3][-1], orbits[0][4][-1], orbits[0][5][-1]]]
    if i == 0:
        xpos_coef = orbits[0][0]
        ypos_coef = orbits[0][1]
        zpos_coef = orbits[0][2]
    else:
        xpos_coef = np.append(xpos_coef, orbits[0][0])
        ypos_coef = np.append(ypos_coef, orbits[0][1])
        zpos_coef = np.append(zpos_coef, orbits[0][2])
expend = t.time()  # time the end of EXP reconstruction
print('exp :', expend-expstart)

scale = 100
# ind = np.arange(0, len(xpos_coef), 1000)
fig_label_fontsize = 'x-large'
fig = plt.figure(dpi=300, layout='constrained')
ax = fig.add_subplot(111)
ax.plot(xpos, ypos, 'r', label='sim')
ax.plot(xpos_coef, ypos_coef, 'b', label='coef')
# ax.scatter(xpos_coef[ind], ypos_coef[ind], c='b', marker='x')
ax.set_xlim(-scale, scale)
ax.set_ylim(-scale, scale)
ax.set_xlabel('X [kpc]', size=fig_label_fontsize)
ax.set_ylabel('Y [kpc]', size=fig_label_fontsize)
ax.tick_params(which='both', direction='in', right=True, top=True, labelsize=fig_label_fontsize)
ax.spines['top'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.set_aspect('equal', 'box')
ax.legend(frameon=False, fontsize=fig_label_fontsize)
# fig.savefig('orbit.pdf')
